In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [19]:
activity = pd.read_csv('../original_data/activity_date_class.csv')
survey_mental_health = pd.read_csv('../original_data/survey_knn_mod_partial/survey_mental_health_1.csv')
grades_numeric = pd.read_csv('../grades_numeric.csv')

In [20]:
overall_columns = ['egoid', 'STAITraitTotal_1', 'CESDOverall_1', 'BAIsum_1', 'STAITraitGroup_1', 'CESDGroup_1', 'BAIgroup_1']

survey_mental_health = survey_mental_health[overall_columns].copy()

In [21]:
grade_columns = ['egoid', 'grade', 'AcademicPeriod']

grades_numeric = grades_numeric[grade_columns].copy()

In [22]:
activity = pd.read_csv('../original_data/activity.csv')
activity.head()

,egoid,datadate,complypercent,meanrate,sdrate,steps,floors,sedentaryminutes,lightlyactiveminutes,fairlyactiveminutes,veryactiveminutes,lowrangemins,fatburnmins,cardiomins,peakmins,lowrangecal,fatburncal,cardiocal,peakcal
0,44869,2015-07-28,91,86.669746,11.979352,4034.0,1.0,1034.0,149.0,2.0,12.0,1211,75,9,4,1364.19900,159.79671,61.841969,35.46954
1,44869,2015-07-29,97,87.975342,8.990446,3702.0,0.0,769.0,153.0,0.0,0.0,1273,103,3,0,1322.14650,262.28818,11.997700,0.00000
2,44869,2015-07-30,100,83.281448,16.863153,3660.0,0.0,795.0,139.0,3.0,18.0,1267,109,15,16,1288.98170,232.89841,77.910057,127.26234
3,44869,2015-07-31,80,91.707787,12.464956,6016.0,13.0,954.0,259.0,0.0,0.0,846,295,2,0,954.04791,629.57526,2.224320,0.00000
4,44869,2015-08-01,78,101.268140,10.464320,9846.0,5.0,1078.0,282.0,24.0,8.0,628,502,0,0,702.88513,1080.83410,0.000000,0.00000


In [23]:
activity = activity[(activity['datadate'].str.startswith('2015'))]
activity = activity.drop(columns = ['datadate'])
activity.shape

(52544, 18)

In [25]:
grades_numeric.dtypes

egoid               int64
grade             float64
AcademicPeriod      int64
dtype: object

In [26]:
print(grades_numeric.shape)
grades_numeric = grades_numeric[(grades_numeric['AcademicPeriod'] < 201600)]
grades_numeric = grades_numeric.drop(columns=['AcademicPeriod'])
grades_numeric.shape

(9970, 3)


(2718, 2)

In [27]:
merged_data = pd.merge(activity, survey_mental_health, on='egoid')
merged_data = pd.merge(merged_data, grades_numeric, on='egoid')

In [28]:
merged_data = pd.get_dummies(merged_data.dropna().drop('egoid', axis=1))

In [29]:
grades = merged_data['grade'].copy()

In [30]:
merged_data = merged_data.drop(columns=['grade'])

In [31]:
regr = RandomForestRegressor(random_state=0)

In [32]:
regr.fit(merged_data, grades)

RandomForestRegressor(random_state=0)

In [ ]:
whole_pred = regr.predict(merged_data)

In [ ]:
w_score = regr.score(merged_data, grades)

In [33]:
importance_tuple = []
for index, column in enumerate(merged_data):
    if column == 'grade':
        continue
    else:
        importance_tuple.append((column, regr.feature_importances_[index]))

In [34]:
importance_tuple_sorted = sorted(importance_tuple, key=lambda tup: tup[1], reverse=True)
importance_tuple_sorted

[('STAITraitTotal_1', 0.2508245594955221),
 ('BAIsum_1', 0.24023019048398458),
 ('CESDOverall_1', 0.23806499272843637),
 ('lowrangecal', 0.03835678796572077),
 ('meanrate', 0.028831729672070903),
 ('sdrate', 0.020742556750171972),
 ('lowrangemins', 0.018801532238484218),
 ('sedentaryminutes', 0.017894115267874565),
 ('lightlyactiveminutes', 0.01680693806382935),
 ('steps', 0.0161006892895445),
 ('fatburncal', 0.015280687891898709),
 ('floors', 0.01422745750333854),
 ('fatburnmins', 0.012247665422941291),
 ('veryactiveminutes', 0.011714978385205068),
 ('fairlyactiveminutes', 0.011667293549292699),
 ('STAITraitGroup_1_anxious', 0.00864342336533907),
 ('complypercent', 0.008536959624787384),
 ('cardiomins', 0.007614140257512989),
 ('cardiocal', 0.0065708668681415386),
 ('CESDGroup_1_Not depressed', 0.005462553026932489),
 ('STAITraitGroup_1_not anxious', 0.005445965150953527),
 ('peakcal', 0.0016786112220066405),
 ('CESDGroup_1_Depressed', 0.0016004299754431971),
 ('BAIgroup_1_moderate', 